<a href="https://colab.research.google.com/github/lyj12h/ai_m/blob/main/pre/temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [ ]:
long_df=pd.read_csv('data/long_data.csv',usecols=['date','datatype', 'value']
).rename(columns={'value' : 'temp_C'}).assign(
date=lambda x: pd.to_datetime(x.date) , temp_F=lambda x : (x.temp_C*9/5)+32
)

long_df.head()

FileNotFoundError: ignored

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
cd /content/drive/MyDrive/pre/

/content/drive/MyDrive/pre


## 2021년도 데이터

파일 하나로 합치기

In [13]:
date = ['05.31-06.06', '06.07-06.13','06.14-06.20','06.21-06.27','06.28-07.04','07.05-07.11','07.12-07.18', '07.19-07.25','07.26-08.01','08.02-08.08','08.09-08.15','08.16-08.22','08.23-08.29','08.30-09.05']

# 빈 딕셔너리 생성
temp_data = {}

# 각 파일을 불러와서 딕셔너리에 저장
for temp in date:
    temp_data[temp] = pd.read_csv(f'2021_temp/S-DoT_NATURE_2021.{temp}.csv',encoding='CP949')

# 모든 데이터를 합치기
all_data = pd.concat(temp_data.values(), ignore_index=True)

# csv 파일로 저장
all_data.to_csv('combined_S-DoT_2021_data.csv', index=False)

In [22]:
df_2021=pd.read_csv('combined_S-DoT_2021_data.csv',sep=',',usecols=[2,6,7,14,25],parse_dates=['등록일자'])
df_2021.head()

,시리얼,기온(℃),상대습도( %),소음(dB),등록일자
0,OC3CL200010,17.1,100.0,55.0,2021-05-31 00:07:00
1,OC3CL200013,16.5,100.0,54.0,2021-05-31 00:07:00
2,OC3CL200015,16.8,100.0,59.0,2021-05-31 00:07:00
3,OC3CL200014,17.1,100.0,57.0,2021-05-31 00:07:00
4,OC3CL200028,16.5,100.0,57.0,2021-05-31 00:07:00


1. 모든 파일에 대해 가져와야함
2. 필요한 6~9월만 선택하고 합친 데이터를 하나 만들어야함

--- 결측치와 이상치 확인 -40 이런 애 체크하기 ---
3. 만들고나면 s-dot 정보와 join해서 s-dot 정보에 있는 시리얼만 남기기 left join?
4. 일평균 항목을 만든다
5. 월평균 항목을 만든다

-> 일단 위 과정을 함수로 만들어야할듯 월 선택하는거까지

In [42]:
#6월 데이터 분류
df_2021_june = df_2021[df_2021['등록일자'].between('2021-06-01', '2021-07-01')]
df_2021_june.rename(columns = {'시리얼':'serial','기온(℃)' : 'temp','상대습도( %)':'humudity','등록일자':'date_time'}, inplace = True)
df_2021_june

<ipython-input-42-22c1778ffbc6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021_june.rename(columns = {'시리얼':'serial','기온(℃)' : 'temp','상대습도( %)':'humudity','등록일자':'date_time'}, inplace = True)


,serial,temp,humudity,소음(dB),date_time
25321,OC3CL200014,20.9,73.0,53.0,2021-06-01 00:07:00
25322,OC3CL200012,21.1,78.0,56.0,2021-06-01 00:07:00
25323,OC3CL200010,16.9,96.0,47.0,2021-06-01 00:07:00
25324,OC3CL200011,21.6,88.0,47.0,2021-06-01 00:07:00
25325,OC3CL200015,20.3,76.0,56.0,2021-06-01 00:07:00
...,...,...,...,...,...
661540,V02Q1940955,25.8,75.0,40.0,2021-06-30 23:07:00
661541,V02Q1941013,26.3,70.0,NaN,2021-06-30 23:07:00
661542,V02Q1941006,25.5,78.0,NaN,2021-06-30 23:07:00
661543,V02Q1941015,26.7,70.0,NaN,2021-06-30 23:07:00


In [43]:
#6월 데이터의 결측치 확인
df_2021_june.isnull().sum()

serial          0
temp          303
humudity      303
소음(dB)       3972
date_time       0
dtype: int64

In [46]:
#시리얼 갯수
n1=df_2021_june['serial'].nunique()
n2=df_2021_july['시리얼'].nunique()
n3=df_2021_aug['시리얼'].nunique()
print(n1,n2,n3)

#8월엔 하나 시리얼이 늘었음

1092 1097 1098


In [30]:
df_2021_june.describe()

,기온(℃),상대습도( %),소음(dB)
count,635921.000000,635921.000000,632252.000000
mean,24.280307,68.000549,48.463415
std,3.644943,15.574913,7.745069
min,-40.000000,0.000000,35.000000
25%,21.800000,57.000000,43.000000
50%,23.900000,69.000000,47.000000
75%,26.700000,79.000000,53.000000
max,40.400000,100.000000,81.000000


In [58]:
#이상한 데이터 확인
check=df_2021_june.loc[df_2021_june.temp<10]
check

,serial,temp,humudity,소음(dB),date_time
579212,OC3CL200210,-4.8,31.0,64.0,2021-06-27 20:07:00
580299,OC3CL200210,-35.4,0.0,62.0,2021-06-27 21:07:00
581381,OC3CL200210,-35.4,0.0,61.0,2021-06-27 22:07:00
582472,OC3CL200210,-35.4,0.0,61.0,2021-06-27 23:07:00
583549,OC3CL200210,-12.6,0.0,60.0,2021-06-28 00:07:00
584627,OC3CL200210,-12.1,0.0,59.0,2021-06-28 01:07:00
585720,OC3CL200210,-12.1,0.0,59.0,2021-06-28 02:07:00
586804,OC3CL200210,-12.1,0.0,58.0,2021-06-28 03:07:00
603114,OC3CL200210,-40.0,0.0,65.0,2021-06-28 18:07:00
604209,OC3CL200210,-9.8,0.0,64.0,2021-06-28 19:07:00


In [26]:
#7월 데이터 분류
df_2021_july = df_2021[df_2021['등록일자'].between('2021-07-01', '2021-08-01')]
df_2021_july

,시리얼,기온(℃),상대습도( %),소음(dB),등록일자
661545,OC3CL200011,21.6,88.0,47.0,2021-07-01 00:07:00
661546,OC3CL200010,21.1,100.0,48.0,2021-07-01 00:07:00
661547,OC3CL200012,25.5,80.0,57.0,2021-07-01 00:07:00
661548,OC3CL200014,25.5,79.0,52.0,2021-07-01 00:07:00
661549,OC3CL200015,25.1,84.0,56.0,2021-07-01 00:07:00
...,...,...,...,...,...
1441213,V02Q1941006,29.4,68.0,NaN,2021-07-31 23:07:00
1441214,V02Q1941000,30.2,66.0,NaN,2021-07-31 23:07:00
1441215,V02Q1940955,29.4,67.0,46.0,2021-07-31 23:07:00
1441216,V02Q1941015,29.8,66.0,NaN,2021-07-31 23:07:00


In [29]:
#8월 데이터 분류
df_2021_aug = df_2021[df_2021['등록일자'].between('2021-08-01', '2021-09-01')]
df_2021_aug.head()

,시리얼,기온(℃),상대습도( %),소음(dB),등록일자
1441218,OC3CL200012,29.9,68.0,57.0,2021-08-01 00:07:00
1441219,OC3CL200016,29.7,68.0,44.0,2021-08-01 00:07:00
1441220,OC3CL200014,29.5,69.0,52.0,2021-08-01 00:07:00
1441221,OC3CL200017,29.1,74.0,60.0,2021-08-01 00:07:00
1441222,OC3CL200018,29.0,69.0,54.0,2021-08-01 00:07:00


##2022년도 데이터

In [18]:
df=pd.read_csv('2022_temp/S-DoT_NATURE_2022.05.30-06.05.csv',encoding='CP949',sep=',',usecols=[2,6,7,14,25],parse_dates=['등록일자'])

df.head()

시리얼  기온(℃)  상대습도( %)  소음(dB)                등록일자
서울시 SDOT001  OC3CL200010   18.5     100.0    38.0 2022-05-30 00:07:01
    SDOT001  OC3CL200012   22.8      60.0    63.0 2022-05-30 00:07:01
    SDOT001  OC3CL200011    NaN       NaN    46.0 2022-05-30 00:07:01
    SDOT001  OC3CL200020   21.7      89.0    67.0 2022-05-30 00:07:01
    SDOT001  OC3CL200026   20.2      83.0    60.0 2022-05-30 00:07:01

In [54]:
df.describe()

,기온(℃),상대습도( %),소음(dB)
count,164578.000000,164578.000000,163958.000000
mean,23.023235,53.663327,48.714896
std,7.831524,17.280286,7.868463
min,-40.000000,11.000000,35.000000
25%,21.100000,44.000000,43.000000
50%,23.500000,53.000000,47.000000
75%,26.400000,66.000000,53.000000
max,38.100000,100.000000,81.000000


In [59]:
df.dtypes

시리얼                 object
기온(℃)              float64
상대습도( %)           float64
소음(dB)             float64
등록일자        datetime64[ns]
dtype: object

In [61]:
df['datetime_m'] = df['등록일자'].dt.strftime('%Y-%m-%d')
df.head()


시리얼  기온(℃)  상대습도( %)  소음(dB)                등록일자  \
서울시 SDOT001  OC3CL200010   18.5     100.0    38.0 2022-05-30 00:07:01   
    SDOT001  OC3CL200012   22.8      60.0    63.0 2022-05-30 00:07:01   
    SDOT001  OC3CL200011    NaN       NaN    46.0 2022-05-30 00:07:01   
    SDOT001  OC3CL200020   21.7      89.0    67.0 2022-05-30 00:07:01   
    SDOT001  OC3CL200026   20.2      83.0    60.0 2022-05-30 00:07:01   

             datetime_m  
서울시 SDOT001  2022-05-30  
    SDOT001  2022-05-30  
    SDOT001  2022-05-30  
    SDOT001  2022-05-30  
    SDOT001  2022-05-30

In [21]:
dataset_filtered = df[df['등록일자'].between('2022-06-01', '2022-06-06')]
dataset_filtered

시리얼  기온(℃)  상대습도( %)  소음(dB)                등록일자  \
서울시 SDOT001  OC3CL200020   21.0      36.0    69.0 2022-06-01 00:07:00   
    SDOT001  OC3CL200022   21.2      31.0    66.0 2022-06-01 00:07:00   
    SDOT001  OC3CL200019   20.4      37.0    59.0 2022-06-01 00:07:00   
    SDOT001  OC3CL200016   21.7      34.0    51.0 2022-06-01 00:07:00   
    SDOT001  OC3CL200013   17.9      45.0    48.0 2022-06-01 00:07:00   
...                  ...    ...       ...     ...                 ...   
    SDOT001  V02Q1940946   21.5      64.0    44.0 2022-06-05 23:07:57   
    SDOT001  V02Q1940948   21.9      63.0    41.0 2022-06-05 23:07:57   
    SDOT001  V02Q1940952   21.5      65.0    39.0 2022-06-05 23:07:57   
    SDOT001  V02Q1940944   21.3      68.0    44.0 2022-06-05 23:07:57   
    SDOT001  V02Q1940945   21.9      64.0    44.0 2022-06-05 23:07:57   

             datetime_m  
서울시 SDOT001  2022-06-01  
    SDOT001  2022-06-01  
    SDOT001  2022-06-01  
    SDOT001  2022-06-01  
    SDOT001  2022-06-01  
...                 ...  
    SDOT001  2022-06-05  
    SDOT001  2022-06-05  
    SDOT001  2022-06-05  
    SDOT001  2022-06-05  
    SDOT001  2022-06-05  

[117653 rows x 6 columns]